In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import heapq


In [2]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Preparing Dataset

In [3]:
skincare_df = pd.read_csv("/content/drive/MyDrive/Capstone Project /Machine Learning/products.csv")
skincare_df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=..."
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f..."


In [4]:
skincare_df['skintype'].value_counts()

,count
skintype,
Oily,310
"Normal, Dry, Oily, Combination, Sensitive",263
Dry,167
"Normal, Dry",93
"Normal, Dry, Oily, Combination",68
Sensitive,55
"Normal, Dry, Oily, Sensitive",52
"Oily, Combination, Sensitive",43
"Dry, Oily, Sensitive",36


In [5]:
skincare_df['concern1'] = skincare_df['notable_effects'].str.split(',').str[0].str.strip()
skincare_df['concern2'] = skincare_df['notable_effects'].str.split(',').str[1].str.strip()
skincare_df['concern3'] = skincare_df['notable_effects'].str.split(',').str[2].str.strip()
skincare_df['concern4'] = skincare_df['notable_effects'].str.split(',').str[3].str.strip()

skincare_df.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,concern1,concern2,concern3,concern4
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,Soothing,Balancing,NaN,NaN
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",Soothing,Balancing,NaN,NaN
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",Brightening,Soothing,NaN,NaN


In [6]:
skincare_df2 = skincare_df.copy()  # Create a copy of skincare_df and assign it to skincare_df2
skincare_df2 = skincare_df2[skincare_df2['skintype'].isna() == False]
skincare_df2.index = [i for i in range(0, len(skincare_df2))]
skincare_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1224 entries, 0 to 1223
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_href     1224 non-null   object
 1   product_name     1224 non-null   object
 2   product_type     1224 non-null   object
 3   brand            1224 non-null   object
 4   notable_effects  1224 non-null   object
 5   skintype         1224 non-null   object
 6   price            1224 non-null   object
 7   description      1224 non-null   object
 8   picture_src      1224 non-null   object
 9   concern1         1224 non-null   object
 10  concern2         1035 non-null   object
 11  concern3         542 non-null    object
 12  concern4         79 non-null     object
dtypes: object(13)
memory usage: 133.9+ KB


In [7]:
skincare_df['product_type'].value_counts()

,count
product_type,
Serum,307
Toner,255
Moisturizer,248
Sunscreen,212
Face Wash,202


In [8]:
unique_effects = []
for effect_list in skincare_df['notable_effects'].str.split(',').dropna():
  if effect_list is not None:
    for effect in effect_list:
      effect = effect.strip()
      if effect not in unique_effects:
        unique_effects.append(effect)

unique_effects

['Acne-Free',
 'Pore-Care',
 'Brightening',
 'Anti-Aging',
 'Soothing',
 'Balancing',
 'Moisturizing',
 'Hydrating',
 'UV-Protection',
 'Oil-Control',
 'No-Whitecast',
 'Skin-Barrier',
 'Refreshing',
 'Black-Spot',
 'Oil-control',
 'Acne-Spot']

In [9]:
unique_skintypes = []
for skin_list in skincare_df['skintype'].str.split(',').dropna():
  if skin_list is not None:
    for skin in skin_list:
      skin = skin.strip()
      if skin not in unique_skintypes:
        unique_skintypes.append(skin)

unique_skintypes

['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive']

In [10]:
skintypes_count = {}
for skin in unique_skintypes:
  skintypes_count[skin] = 0
  for skin_list in skincare_df['skintype'].str.split(','):
    if skin_list is not None:
      for current_skin in skin_list:
        current_skin = current_skin.strip()
        if current_skin == skin:
          skintypes_count[skin] += 1
          break

for skin, count in skintypes_count.items():
  print(f"{skin}: {count}")

Oily: 839
Normal: 538
Dry: 730
Combination: 441
Sensitive: 500


In [11]:
EFFECTS = list(skincare_df2.concern1.unique())
print(EFFECTS)

['Acne-Free', 'Soothing', 'Brightening', 'Moisturizing', 'Hydrating', 'Pore-Care', 'Anti-Aging', 'Balancing', 'UV-Protection', 'Skin-Barrier', 'Refreshing', 'Oil-Control']


In [12]:
PRODUCTS = list(skincare_df2.product_type.unique())
print(PRODUCTS)

['Face Wash', 'Toner', 'Serum', 'Moisturizer', 'Sunscreen']


In [13]:
skincare_df2[skincare_df2['product_type'] ==  'Face Wash']['concern1'].value_counts()

,count
concern1,
Acne-Free,45
Hydrating,44
Moisturizing,34
Soothing,22
Pore-Care,19
Anti-Aging,17
Refreshing,7
Brightening,5
Balancing,4


In [14]:
skincare_df2[skincare_df2['product_type'] == 'Sunscreen']['concern1'].value_counts()

,count
concern1,
UV-Protection,57
Brightening,33
Anti-Aging,32
Moisturizing,29
Hydrating,23
Soothing,16
Acne-Free,12
Skin-Barrier,5
Pore-Care,5


In [15]:
skincare_df2['notable_effects'] = skincare_df2['notable_effects'].str.lower()

In [16]:
top_concerns = {'Face Wash':'Refreshing', 'Toner':'Soothing', 'Serum':'Brightening', 'Moisturizer':'Moisturizing', 'Sunscreen':'UV-Protection' }
entries = len(skincare_df2)
for i in range(entries):
    product_type = skincare_df2.iloc[i]['product_type']
    if pd.isnull(skincare_df2.iloc[i]['notable_effects']):
        skincare_df2.iloc[i]['notable_effects'] = top_concerns[product_type]
skincare_df2.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1224 entries, 0 to 1223
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   product_href     1224 non-null   object
 1   product_name     1224 non-null   object
 2   product_type     1224 non-null   object
 3   brand            1224 non-null   object
 4   notable_effects  1224 non-null   object
 5   skintype         1224 non-null   object
 6   price            1224 non-null   object
 7   description      1224 non-null   object
 8   picture_src      1224 non-null   object
 9   concern1         1224 non-null   object
 10  concern2         1035 non-null   object
 11  concern3         542 non-null    object
 12  concern4         79 non-null     object
dtypes: object(13)
memory usage: 133.9+ KB


In [17]:
skincare_df2['brand'] = skincare_df2['brand'].str.lower()
skincare_df2['product_name'] = skincare_df2['product_name'].str.lower()
skincare_df2['skintype'] = skincare_df2['skintype'].str.lower()
skincare_df2['notable_effects'] = skincare_df2['notable_effects'].str.replace(' and ', ',').str.replace(' or ', ',')
skincare_df2

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,concern1,concern2,concern3,concern4
0,https://www.beautyhaul.com/product/detail/bubb...,acwell bubble free ph balancing cleanser,Face Wash,acwell,"acne-free, pore-care, brightening, anti-aging",oily,Rp 209.000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
1,https://www.sociolla.com/face-wash/62452-ph-ba...,acwell ph balancing soothing cleansing foam,Face Wash,acwell,"soothing, balancing","normal, dry, combination",Rp 181.800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,Soothing,Balancing,NaN,NaN
2,https://www.sociolla.com/toner/15871-licorice-...,acwell licorice ph balancing cleansing toner,Toner,acwell,"soothing, balancing","normal, dry, oily, combination, sensitive",Rp 149.000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",Soothing,Balancing,NaN,NaN
3,https://www.beautyhaul.com/product/detail/aqua...,acwell aquaseal soothing tonic,Toner,acwell,"acne-free, pore-care, brightening, anti-aging",oily,Rp 290.000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,Acne-Free,Pore-Care,Brightening,Anti-Aging
4,https://www.sociolla.com/essence/38023-licoric...,licorice ph balancing essence mist,Toner,acwell,"brightening, soothing","normal, dry",Rp 194.650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",Brightening,Soothing,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,https://www.sociolla.com/face-cream-lotion/263...,whitelab brightening day cream,Moisturizer,whitelab,"brightening, uv-protection","normal, dry, oily, combination, sensitive",Rp 61.950,Whitelab Brightening Day Cream Fungsi: - Memba...,"https://www.soco.id/cdn-cgi/image/w=73,format=...",Brightening,UV-Protection,NaN,NaN
1220,https://shopee.co.id/YOU-The-Radiance-White-Pu...,you the radiance white purifying facial foam,Face Wash,you,"moisturizing, refreshing","normal, dry, oily, sensitive",Rp 52.000,YOU The Radiance White Purifying Facial Foam [...,https://cdn.istyle.im/images/product/web/27/61...,Moisturizing,Refreshing,NaN,NaN
1221,https://shopee.co.id/YOU-Basic-Skin-Care-Simpl...,you basic skin care simply fresh and bright fa...,Face Wash,you,"refreshing, acne-free","normal, dry, oily, sensitive",Rp 25.000,YOU Simply Fresh Honey and lemon Facial Wash S...,https://images.tokopedia.net/img/cache/700/pro...,Refreshing,Acne-Free,NaN,NaN
1222,https://shopee.co.id/YOU-Dazzling-Glow-Up-Faci...,you dazzling glow up facial foam,Face Wash,you,"hydrating, moisturizing","normal, dry, oily, sensitive",Rp 35.000,"A mild facial foam with Rice Extract, Tranexam...",https://images.tokopedia.net/img/cache/700/Vqb...,Hydrating,Moisturizing,NaN,NaN


# Concerns and Features

In [44]:
concerns = ['acne', 'blackhead', 'pores', 'redness', 'wrinkles', 'darkspot']
print(concerns)

['acne', 'blackhead', 'pores', 'redness', 'wrinkles', 'darkspot']


In [45]:
features = list(unique_skintypes) + list(concerns)
(features)

features = ['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive', 'acne', 'blackhead', 'pores', 'redness', 'wrinkles', 'darkspot']

# Recomendation Model

In [46]:
# Step 1: Define concern mapping
concern_mapping = {
    'acne': ['Acne-Free', 'Acne-Spot'],
    'comedo': ['Pore-Care', 'Oil-Control'],
    'pores': ['Pore-Care'],
    'darkspot': ['Brightening', 'Black-Spot'],
    'wrinkles': ['Anti-Aging'],
    'redness': ['Soothing', 'Balancing', 'Refreshing'],
    'Dry': ['Moisturizing', 'Hydrating'],
    'Combination': ['Moisturizing'],
    'Oily': ['Oil-Control', 'Oil-control'],
    'Normal': ['Hydrating', 'Skin-Barrier'],
    'Sensitive': ['Skin-Barrier']
}

In [49]:
# Step 2: Prepare the features (One-Hot Encoding)
# List of unique features (skin types + concerns)
features = ['Oily', 'Normal', 'Dry', 'Combination', 'Sensitive',
            'acne', 'blackhead', 'pores', 'redness', 'wrinkles', 'darkspot']

# Create an empty DataFrame for one-hot encoding
encoded_features = pd.DataFrame(0, index=skincare_df.index, columns=features)

# Populate one-hot encoding based on skintype column
for index, row in skincare_df.iterrows():
    # Split skintype into individual values and set them to 1
    for skintype in row['skintype'].split(', '):
        if skintype in features:
            encoded_features.at[index, skintype] = 1

In [50]:
# Step 3: Input skin condition
input_skin_condition = ['Oily', 'redness', 'darkspot']

# Generate a one-hot encoded vector for the input skin condition
input_vector = [1 if feature in input_skin_condition else 0 for feature in features]

In [51]:
# Step 4: Map input skin condition to notable effects
recommended_effects = set()
for condition in input_skin_condition:
    if condition in concern_mapping:
        recommended_effects.update(concern_mapping[condition])

# Normalize recommended effects for consistent comparison
recommended_effects = {effect.lower() for effect in recommended_effects}

In [52]:
# Step 5: Filter products based on skintype and notable effects
def filter_products(row):
    # Check skintype match
    product_skintypes = set(row['skintype'].split(', '))
    if not any(st in input_skin_condition for st in product_skintypes):
        return False  # Skintype does not match

    # Check if product satisfies all recommended effects
    product_effects = {effect.lower() for effect in row['notable_effects'].split(', ')}
    return recommended_effects.issubset(product_effects)

recommended_products = skincare_df[skincare_df.apply(filter_products, axis=1)]


In [53]:
# Step 6: Output the recommended products
output_columns = ['product_href', 'product_name', 'price', 'brand','skintype','notable_effects']
recommended_products_output = recommended_products[output_columns]


In [54]:
def search_concern(target, i):
    if target in skincare_df2.iloc[i]['notable_effect']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        skintype = skincare_df2.iloc[i]['skintype']
        if skintype == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == skintype:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in skincare_df2.iloc[i]['concern1']:
            one_hot_encodings[i][j] = 1



In [55]:
x = one_hot_encodings[456]

In [56]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)


In [57]:
# utility functions
def name2index(name):
    return skincare_df2[skincare_df2["product_name"]==name].index.tolist()[0]

def index2prod(index):
    return skincare_df2.iloc[index]

In [58]:
def wrap(info_arr):
    result = {}
#   print(info_arr)
    result['product_href'] = info_arr[0]
    result['product_name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['brand'] = info_arr[3]
    result['skintype'] = info_arr[4]
    result['notable_effects'] = str(info_arr[5]).split(',')
    return result


# recommend top 2 similar items from a category
def recs_cs(vector=None, product_name=None, product_type=None, count=2):
    products = []
    if product_name:
        idx = name2index(product_name)
        fv = one_hot_encodings[idx]
    elif vector:
        # Ensure the vector has the same number of features as one_hot_encodings
        fv = vector[:one_hot_encodings.shape[1]]  # Truncate or pad if necessary
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    skincare_df2['cs'] = cs_values[0]

    if product_type:
        skincare_dff = skincare_df2[skincare_df2['product_type'] == product_type]
    else:
        skincare_dff = skincare_df2

    if product_name:
        skincare_dff = skincare_dff[skincare_dff['product_name'] != product_name]
    recommendations = skincare_dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['product_href', 'product_name', 'price', 'brand','skintype','notable_effects']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [59]:
# overall recommendation
def recs_essentials(vector = None, product_name = None):
#     print("ESSENTIALS:")
    response = {}
    for product in PRODUCTS:
#         print(f"{label}:")
        if product_name:
            r = recs_cs(None, product_name, product_type)
        elif vector:
            r = recs_cs(vector, None, product_type)
        else:
            r = recs_cs(None, None, product_type)
        response[product_type] = r
    return response


In [60]:
x = [0,1,0,0,0,1,0,1,0,0,0]

y1 = recs_cs(x, None, 'Face Wash')
y2 = recs_cs(x, None, 'Toner')
y3 = recs_cs(x, None, 'Serum')
y4 = recs_cs(x, None, 'Moisturizer')
y5 = recs_cs(x, None, 'Sunscreen')

y1

[{'product_href': 'https://www.beautyhaul.com/product/detail/bubble-free-ph-balancing-cleanser',
  'product_name': 'acwell bubble free ph balancing cleanser',
  'price': 'Rp 209.000',
  'brand': 'acwell ',
  'skintype': 'oily',
  'notable_effects': ['acne-free',
   ' pore-care',
   ' brightening',
   ' anti-aging']},
 {'product_href': 'https://www.zalora.co.id/sk-ii-sk-ii-cleansing-gel-80gr-white-2975877.html',
  'product_name': 'sk-ii cleansing gel 80gr',
  'price': 'Rp 540.000',
  'brand': 'sk-ii',
  'skintype': 'oily, combination',
  'notable_effects': ['acne-free', ' oil-control', ' pore-care']}]

In [61]:
y2

[{'product_href': 'https://www.sociolla.com/toner/15871-licorice-ph-balancing-cleansing-toner',
  'product_name': 'acwell licorice ph balancing cleansing toner ',
  'price': 'Rp 149.000',
  'brand': 'acwell ',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['soothing', ' balancing']},
 {'product_href': 'https://www.beautyhaul.com/product/detail/cica-clear-pad',
  'product_name': 'npure cica clear pad',
  'price': 'Rp 129.000',
  'brand': "n'pure",
  'skintype': 'dry, oily, sensitive',
  'notable_effects': ['brightening', ' anti-aging']}]

In [62]:
y3

[{'product_href': 'https://www.sociolla.com/face-serum/54021-licorice-ph-balancing-advance-serum ',
  'product_name': 'licorice ph balancing advance serum',
  'price': 'Rp 187.000',
  'brand': 'acwell ',
  'skintype': 'normal, dry, oily, combination',
  'notable_effects': ['moisturizing', ' brightening', ' anti-aging']},
 {'product_href': 'https://www.beautyhaul.com/product/detail/eternal-intensive-serum',
  'product_name': 'skeyndor eternal intensive serum',
  'price': 'Rp 1.045.000',
  'brand': 'skeyndor ',
  'skintype': 'normal, dry, oily, sensitive',
  'notable_effects': ['brightening', ' anti-aging']}]

In [63]:
y4

[{'product_href': 'https://www.sociolla.com/face-cream-lotion/71655-licorice-ph-balancing-essential-cream',
  'product_name': 'licorice ph balancing essential cream',
  'price': 'Rp 304.000',
  'brand': 'acwell ',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['moisturizing', ' brightening']},
 {'product_href': 'https://www.sociolla.com/face-cream-lotion/25445-sea-buckthorn-vital-70-cream',
  'product_name': 'purito sea buckthorn vital 70 cream ',
  'price': 'Rp 412.500',
  'brand': 'purito',
  'skintype': 'normal, dry, oily, combination, sensitive',
  'notable_effects': ['soothing', ' moisturizing']}]

In [64]:
y5

[{'product_href': 'https://www.beautyhaul.com/product/detail/sunorion-uv-sunscreen',
  'product_name': 'aizen sunorion uv sunscreen',
  'price': 'Rp 139.000',
  'brand': 'aizen ',
  'skintype': 'dry, oily, sensitive',
  'notable_effects': ['anti-aging', ' uv-protection']},
 {'product_href': 'https://www.beautyhaul.com/product/detail/uv-defender-matte-fresh',
  'product_name': 'loreal paris uv defender matte & fresh ',
  'price': 'Rp 319.000',
  'brand': 'loreal paris',
  'skintype': 'dry',
  'notable_effects': ['anti-aging', ' uv-protection']}]

In [65]:
x = [1,0,0,0,0,0,1,1,0,0,0]

y = recs_essentials(x, None,)
y

{'Face Wash': [{'product_href': 'https://www.beautyhaul.com/product/detail/bubble-free-ph-balancing-cleanser',
   'product_name': 'acwell bubble free ph balancing cleanser',
   'price': 'Rp 209.000',
   'brand': 'acwell ',
   'skintype': 'oily',
   'notable_effects': ['acne-free',
    ' pore-care',
    ' brightening',
    ' anti-aging']},
  {'product_href': 'https://www.zalora.co.id/sk-ii-sk-ii-cleansing-gel-80gr-white-2975877.html',
   'product_name': 'sk-ii cleansing gel 80gr',
   'price': 'Rp 540.000',
   'brand': 'sk-ii',
   'skintype': 'oily, combination',
   'notable_effects': ['acne-free', ' oil-control', ' pore-care']}]}

In [66]:
def filter_products(row):
    # Memeriksa apakah skin type cocok
    product_skintypes = set(row['skintype'].split(', '))
    if not any(st.lower() in input_skin_condition for st in product_skintypes):
        return False  # Skin type tidak cocok

    # Memeriksa apakah semua efek yang direkomendasikan terpenuhi
    product_effects = {effect.lower() for effect in row['notable_effects'].split(', ')}
    return recommended_effects.issubset(product_effects)


In [67]:
def recommend_products(input_conditions, skincare_df, concern_mapping):
    # Extract skin type and concerns from input
    skin_type = input_conditions[0].lower()
    concerns = {cond.lower() for cond in input_conditions[1:]}

    # Filter based on skin type
    filtered_df = skincare_df[skincare_df['skintype'].str.contains(skin_type, case=False, na=False)]

    # Generate a list of recommended effects based on concerns
    recommended_effects = set()
    for concern in concerns:
        if concern in concern_mapping:
            recommended_effects.update({effect.lower() for effect in concern_mapping[concern]})

    # Function to filter products based on notable effects
    def match_effects(effects):
        product_effects = {effect.lower() for effect in effects.split(', ')}
        return not recommended_effects.isdisjoint(product_effects)

    # Further filter the DataFrame based on notable effects
    filtered_df = filtered_df[filtered_df['notable_effects'].apply(match_effects)]

    # Select one product per product type
    representative_products = (
        filtered_df.groupby('product_type', as_index=False)
        .apply(lambda group: group.iloc[0])  # Take the first product in each group
        .reset_index(drop=True)
    )

    # Columns to display in the output
    output_columns = ['product_name', 'brand', 'product_type', 'price', 'skintype',
                      'concern1', 'concern2', 'concern3', 'concern4']

    return representative_products[output_columns]

In [71]:
test_input = ['Dry', 'acne']
recommendations = recommend_products(test_input, skincare_df, concern_mapping)
print(recommendations if not recommendations.empty else "No products found.")


                                        product_name        brand  \
0  BIO ESSENCE White Advanced Whitening Cleanser ...  BIO ESSENCE   
1                    ARGAVELL 100% Organic Argan Oil     ARGAVELL   
2  AVOSKIN YOUR SKIN BAE SERIES Lactic Acid 10% +...      AVOSKIN   
3                 Bioderma Photoderm AKN Mat SPF-30      BIODERMA   
4            AVOSKIN Miraculous Refining Toner 100ml      AVOSKIN   

  product_type       price                                   skintype  \
0    Face Wash   Rp 49.000               Normal, Dry, Oily, Sensitive   
1  Moisturizer  Rp 130.000             Normal, Dry, Oily, Combination   
2        Serum  Rp 122.550             Normal, Dry, Oily, Combination   
3    Sunscreen  Rp 319.000  Normal, Dry, Oily, Combination, Sensitive   
4        Toner  Rp 189.000                       Dry, Oily, Sensitive   

    concern1     concern2     concern3      concern4  
0  Acne-Free  Oil-Control    Pore-Care           NaN  
1  Acne-Free  Oil-Control   Anti-Agi

<ipython-input-67-2701311572a7>:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.iloc[0])  # Take the first product in each group
